In [1]:
import os
import pandas as pd
import numpy as np
from pathlib import Path
import json
import warnings
warnings.filterwarnings('ignore')

print("🚀 Simple Kinyarwanda TTS Setup")
print("================================")

# Load your dataset
def load_kinyarwanda_words():
    """Load Kinyarwanda words from your dataset"""
    
    # Load from TTS CSV
    tts_data_path = "../datasets/tts_data.csv"
    if os.path.exists(tts_data_path):
        df = pd.read_csv(tts_data_path)
        words = df['text'].unique().tolist()
        print(f"✅ Loaded {len(words)} unique words from TTS dataset")
        return words
    
    # Fallback: manual list from your dataset
    fallback_words = [
        "amakuru mashya", "zeru", "murwanda", "kuvugisha ubishinzwe", "gatanu",
        "kwipima", "ingendo", "ibihano namande", "ibiciro", "ibibujijwe",
        "gatandatu", "amabwiriza", "rimwe", "kabiri", "icyenda",
        "amasaha ntarengwa", "umunani", "gumamurugo", "oya", "abaturanyi",
        "Muraho", "Mwaramutse", "Amakuru", "Murakoze", "Ni meza cyane"
    ]
    print(f"✅ Using fallback word list: {len(fallback_words)} words")
    return fallback_words

# Load words
kinyarwanda_words = load_kinyarwanda_words()
print(f"\n📝 Sample words: {kinyarwanda_words[:5]}")
print(f"📊 Total words to process: {len(kinyarwanda_words)}")


🚀 Simple Kinyarwanda TTS Setup
✅ Loaded 31 unique words from TTS dataset

📝 Sample words: ['amakuru mashya', 'zeru', 'murwanda', 'kuvugisha ubishinzwe', 'gatanu']
📊 Total words to process: 31


In [2]:
# Setup Free TTS (no licensing issues)
def setup_free_tts():
    """Setup free TTS models for audio generation"""
    
    try:
        from TTS.api import TTS
        
        # Free models (no licensing issues)
        free_models = [
            "tts_models/en/ljspeech/tacotron2-DDC",
            "tts_models/en/ljspeech/glow-tts",
            "tts_models/en/ljspeech/speedy-speech"
        ]
        
        print("🔄 Testing free TTS models...")
        
        for model_name in free_models:
            try:
                print(f"  Testing {model_name}...")
                tts = TTS(model_name=model_name)
                print(f"  ✅ {model_name} loaded successfully!")
                return tts, model_name
                
            except Exception as e:
                print(f"  ⚠️ {model_name} failed: {e}")
                continue
        
        return None, None
        
    except ImportError:
        print("⚠️ TTS package not available. Installing gTTS as fallback...")
        
        try:
            import subprocess
            import sys
            subprocess.check_call([sys.executable, "-m", "pip", "install", "gTTS"])
            
            from gtts import gTTS
            print("✅ gTTS installed and ready!")
            return "gTTS", "gTTS"
            
        except Exception as e:
            print(f"❌ All TTS options failed: {e}")
            return None, None

# Kinyarwanda pronunciation improvements
def improve_pronunciation(text):
    """Apply Kinyarwanda phonetic rules to improve pronunciation"""
    
    # Basic Kinyarwanda pronunciation rules
    pronunciation_rules = {
        'rw': 'ru',  # 'rw' sounds like 'ru'
        'ny': 'ni',  # 'ny' sounds like 'ni' 
        'cy': 'chi', # 'cy' sounds like 'chi'
        'by': 'bi',  # 'by' sounds like 'bi'
        'sh': 'sh',  # keep 'sh' as is
        'nk': 'nk',  # keep 'nk' as is
    }
    
    improved_text = text.lower()
    
    for kinyarwanda, english_approx in pronunciation_rules.items():
        improved_text = improved_text.replace(kinyarwanda, english_approx)
    
    return improved_text

# Setup TTS
tts_model, model_name = setup_free_tts()

if tts_model:
    print(f"\n🎉 TTS ready using: {model_name}")
else:
    print("\n❌ No TTS models available")

# Test pronunciation improvement
test_word = "Muraho"
improved = improve_pronunciation(test_word)
print(f"\n🧪 Pronunciation test:")
print(f"  Original: {test_word}")
print(f"  Improved: {improved}")


🔄 Testing free TTS models...
  Testing tts_models/en/ljspeech/tacotron2-DDC...
 > tts_models/en/ljspeech/tacotron2-DDC is already downloaded.
 > vocoder_models/en/ljspeech/hifigan_v2 is already downloaded.
 > Using model: Tacotron2
 > Setting up Audio Processor...
 | > sample_rate:22050
 | > resample:False
 | > num_mels:80
 | > log_func:np.log
 | > min_level_db:-100
 | > frame_shift_ms:None
 | > frame_length_ms:None
 | > ref_level_db:20
 | > fft_size:1024
 | > power:1.5
 | > preemphasis:0.0
 | > griffin_lim_iters:60
 | > signal_norm:False
 | > symmetric_norm:True
 | > mel_fmin:0
 | > mel_fmax:8000.0
 | > pitch_fmin:1.0
 | > pitch_fmax:640.0
 | > spec_gain:1.0
 | > stft_pad_mode:reflect
 | > max_norm:4.0
 | > clip_norm:True
 | > do_trim_silence:True
 | > trim_db:60
 | > do_sound_norm:False
 | > do_amp_to_db_linear:True
 | > do_amp_to_db_mel:True
 | > do_rms_norm:False
 | > db_level:None
 | > stats_path:None
 | > base:2.718281828459045
 | > hop_length:256
 | > win_length:1024
 > Model's 

In [3]:
# Audio generation function
def generate_audio_for_word(word, tts_model, model_name, output_dir="../models/kinyarwanda-tts/audio"):
    """Generate audio for a single Kinyarwanda word"""
    
    os.makedirs(output_dir, exist_ok=True)
    
    # Improve pronunciation
    improved_word = improve_pronunciation(word)
    
    # Generate filename
    safe_filename = word.replace(" ", "_").replace("/", "_")
    output_path = os.path.join(output_dir, f"{safe_filename}.wav")
    
    try:
        if model_name == "gTTS":
            # Use Google TTS
            from gtts import gTTS
            tts = gTTS(text=improved_word, lang='en', slow=False)
            tts.save(output_path.replace('.wav', '.mp3'))
            output_path = output_path.replace('.wav', '.mp3')
            
        else:
            # Use Coqui TTS
            tts_model.tts_to_file(text=improved_word, file_path=output_path)
        
        if os.path.exists(output_path):
            return output_path
        else:
            return None
            
    except Exception as e:
        print(f"❌ Failed to generate audio for '{word}': {e}")
        return None

# Generate audio for a few test words
def generate_test_audio(words, tts_model, model_name, max_words=5):
    """Generate audio for test words"""
    
    if not tts_model:
        print("❌ No TTS model available")
        return []
    
    test_words = words[:max_words]  # Just test with first few words
    print(f"\n🎵 Generating audio for {len(test_words)} test words...")
    
    results = []
    
    for i, word in enumerate(test_words, 1):
        print(f"  [{i}/{len(test_words)}] Processing: '{word}'")
        
        audio_path = generate_audio_for_word(word, tts_model, model_name)
        
        if audio_path:
            print(f"    ✅ Generated: {audio_path}")
            results.append({
                "word": word,
                "audio_path": audio_path,
                "improved_pronunciation": improve_pronunciation(word)
            })
        else:
            print(f"    ❌ Failed: {word}")
    
    print(f"\n🎉 Successfully generated audio for {len(results)} words!")
    return results

# Generate audio for test words
if tts_model:
    audio_results = generate_test_audio(kinyarwanda_words, tts_model, model_name, max_words=5)
    
    # Save results
    results_path = "../models/kinyarwanda-tts/audio_results.json"
    os.makedirs(os.path.dirname(results_path), exist_ok=True)
    with open(results_path, 'w', encoding='utf-8') as f:
        json.dump(audio_results, f, ensure_ascii=False, indent=2)
    
    print(f"\n📄 Results saved to: {results_path}")
else:
    print("\n❌ Cannot generate audio - no TTS model available")
    audio_results = []



🎵 Generating audio for 5 test words...
  [1/5] Processing: 'amakuru mashya'
 > Text splitted to sentences.
['amakuru mashya']
   > Decoder stopped with `max_decoder_steps` 10000
 > Processing time: 114.51201510429382
 > Real-time factor: 0.9815086657064087
    ✅ Generated: ../models/kinyarwanda-tts/audio/amakuru_mashya.wav
  [2/5] Processing: 'zeru'
 > Text splitted to sentences.
['zeru']
   > Decoder stopped with `max_decoder_steps` 10000
 > Processing time: 118.59805607795715
 > Real-time factor: 1.0165310572033133
    ✅ Generated: ../models/kinyarwanda-tts/audio/zeru.wav
  [3/5] Processing: 'murwanda'
 > Text splitted to sentences.
['muruanda']
   > Decoder stopped with `max_decoder_steps` 10000
 > Processing time: 116.4544289112091
 > Real-time factor: 0.9981575385966356
    ✅ Generated: ../models/kinyarwanda-tts/audio/murwanda.wav
  [4/5] Processing: 'kuvugisha ubishinzwe'
 > Text splitted to sentences.
['kuvugisha ubishinzwe']
 > Processing time: 1.6060240268707275
 > Real-time 

In [4]:
# Create simple TTS API for your chatbot
api_code = '''
# Simple Kinyarwanda TTS API
# Generated from simple_kinyarwanda_tts.ipynb

import os
import json

class SimpleKinyarwandaTTS:
    def __init__(self, audio_dir="audio", results_file="audio_results.json"):
        self.audio_dir = audio_dir
        self.results_file = results_file
        self.audio_cache = self._load_audio_cache()
        
    def _load_audio_cache(self):
        """Load pre-generated audio cache"""
        if os.path.exists(self.results_file):
            with open(self.results_file, 'r', encoding='utf-8') as f:
                results = json.load(f)
            return {item['word']: item['audio_path'] for item in results}
        return {}
    
    def get_audio_for_word(self, word):
        """Get audio file path for a Kinyarwanda word"""
        
        # Check cache first
        if word in self.audio_cache:
            audio_path = self.audio_cache[word]
            if os.path.exists(audio_path):
                return audio_path
        
        # Check for file directly
        safe_filename = word.replace(" ", "_").replace("/", "_")
        
        for ext in ['.wav', '.mp3']:
            audio_path = os.path.join(self.audio_dir, f"{safe_filename}{ext}")
            if os.path.exists(audio_path):
                return audio_path
        
        return None
    
    def get_audio_for_text(self, text):
        """Get audio for text (tries to find individual words)"""
        words = text.split()
        audio_files = []
        
        for word in words:
            audio_path = self.get_audio_for_word(word)
            if audio_path:
                audio_files.append(audio_path)
        
        # For now, return the first audio file found
        # Later you could combine multiple audio files
        return audio_files[0] if audio_files else None
    
    def list_available_words(self):
        """List all words with available audio"""
        return list(self.audio_cache.keys())

# Example usage:
# tts = SimpleKinyarwandaTTS()
# audio_path = tts.get_audio_for_word("Muraho")
# if audio_path:
#     print(f"Audio available at: {audio_path}")
# else:
#     print("No audio available for this word")
'''

# Save API code
api_path = "../models/kinyarwanda-tts/simple_tts_api.py"
os.makedirs(os.path.dirname(api_path), exist_ok=True)

with open(api_path, 'w', encoding='utf-8') as f:
    f.write(api_code)

print(f"✅ Simple TTS API created: {api_path}")

# Final summary
print("\n🎯 SIMPLE KINYARWANDA TTS COMPLETE!")
print("="*50)

print(f"\n📊 Results:")
print(f"  Total words processed: {len(kinyarwanda_words)}")
print(f"  Audio files generated: {len(audio_results) if audio_results else 0}")
print(f"  TTS model used: {model_name if tts_model else 'None'}")

print(f"\n📁 Generated Files:")
print(f"  Audio directory: ../models/kinyarwanda-tts/audio/")
print(f"  Results file: ../models/kinyarwanda-tts/audio_results.json")
print(f"  API file: ../models/kinyarwanda-tts/simple_tts_api.py")

print(f"\n🔗 Integration with Your Chatbot:")
print(f"```python")
print(f"from models.kinyarwanda_tts.simple_tts_api import SimpleKinyarwandaTTS")
print(f"")
print(f"# Initialize TTS")
print(f"tts = SimpleKinyarwandaTTS()")
print(f"")
print(f"# Get audio for a word")
print(f"audio_path = tts.get_audio_for_word('Muraho')")
print(f"if audio_path:")
print(f"    print(f'Play audio: {{audio_path}}')")
print(f"```")

print(f"\n📋 Next Steps:")
print(f"1. ✅ Audio files are ready for your chatbot")
print(f"2. ✅ Import the simple_tts_api.py in your chatbot")
print(f"3. ✅ Use get_audio_for_word() to get audio files")
print(f"4. 🔄 Run this notebook again to process more words")
print(f"5. 🎵 Test the audio files to ensure quality")

if audio_results:
    print(f"\n🎉 SUCCESS! Your simple Kinyarwanda TTS is ready!")
    print(f"\n📝 Sample generated files:")
    for result in audio_results[:3]:
        print(f"  '{result['word']}' -> {result['audio_path']}")
        
else:
    print(f"\n⚠️ No audio was generated. Please check TTS setup and try again.")

print(f"\n" + "="*50)


✅ Simple TTS API created: ../models/kinyarwanda-tts/simple_tts_api.py

🎯 SIMPLE KINYARWANDA TTS COMPLETE!

📊 Results:
  Total words processed: 31
  Audio files generated: 5
  TTS model used: tts_models/en/ljspeech/tacotron2-DDC

📁 Generated Files:
  Audio directory: ../models/kinyarwanda-tts/audio/
  Results file: ../models/kinyarwanda-tts/audio_results.json
  API file: ../models/kinyarwanda-tts/simple_tts_api.py

🔗 Integration with Your Chatbot:
```python
from models.kinyarwanda_tts.simple_tts_api import SimpleKinyarwandaTTS

# Initialize TTS
tts = SimpleKinyarwandaTTS()

# Get audio for a word
audio_path = tts.get_audio_for_word('Muraho')
if audio_path:
    print(f'Play audio: {audio_path}')
```

📋 Next Steps:
1. ✅ Audio files are ready for your chatbot
2. ✅ Import the simple_tts_api.py in your chatbot
3. ✅ Use get_audio_for_word() to get audio files
4. 🔄 Run this notebook again to process more words
5. 🎵 Test the audio files to ensure quality

🎉 SUCCESS! Your simple Kinyarwanda TTS 